# Lets import some things

In [87]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torchvision
from torchvision import datasets
import torchvision.transforms.v2 as transforms


# Decide if cuda

In [88]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# Load dataset

In [89]:
batchsize = 16
num_classes = 102
learning_rate = 0.01
num_epochs = 50

In [90]:
trainingData = datasets.Flowers102(
    root="data",
    split = "train",
    download = True,
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor()
    ])
)
testData = datasets.Flowers102(
    root="data",
    split= "test",
    download = True,
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor()
    ])
)
valData = datasets.Flowers102(
    root="data",
    split = "val",
    download = True,
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor()
    ])
)

# Get some dataloaders

In [91]:
train_dataloader = DataLoader(trainingData, batch_size=batchsize, shuffle=True, num_workers=2)
test_dataloader = DataLoader(testData, batch_size=batchsize, shuffle=True,num_workers=2)
val_dataloader = DataLoader(valData, batch_size=batchsize, shuffle=True,num_workers=2)

#todo get some dataloaders of the form compose

# Neural Network class

In [92]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3,32,kernel_size=5,stride=1,padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32,64,kernel_size=5,stride=1,padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(64*64*64 ,1024),
            nn.ReLU(),
            nn.Linear(1024,102),
        )
        
        
    def forward(self, x):
        x= self.features(x)
        x= x.view(x.size(0),-1)
        x = self.classifier(x)
        return x
#conv way selected, this is a uhhh LeNet5 for MNIST so it doesnt work like yeah of course pretty placeholder

# todo train/test/possibly do the image display thing/ save&load models/ https://pytorch.org/tutorials/beginner/introyt/introyt1_tutorial.html / hyperparams

# Model = something

In [93]:
model = NeuralNet().to(device)
cost = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
total_step = len(train_dataloader)

In [94]:
for epoch in range(num_epochs):
    for i, (images,labels) in enumerate(train_dataloader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = cost(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

        print(f'Epoch: [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')
        


Epoch: [1/50], Loss: 4.6307
Epoch: [1/50], Loss: 2310.3237
Epoch: [1/50], Loss: 372.8550
Epoch: [1/50], Loss: 5.6671
Epoch: [1/50], Loss: 23.6528
Epoch: [1/50], Loss: 37.7071
Epoch: [1/50], Loss: 9.5525
Epoch: [1/50], Loss: 9.7136
Epoch: [1/50], Loss: 5.6027
Epoch: [1/50], Loss: 5.5450
Epoch: [1/50], Loss: 5.0146
Epoch: [1/50], Loss: 5.1268
Epoch: [1/50], Loss: 4.7543
Epoch: [1/50], Loss: 4.6436
Epoch: [1/50], Loss: 4.6276
Epoch: [1/50], Loss: 4.7006
Epoch: [1/50], Loss: 4.6384
Epoch: [1/50], Loss: 4.6277
Epoch: [1/50], Loss: 4.6797
Epoch: [1/50], Loss: 4.6333
Epoch: [1/50], Loss: 4.6219
Epoch: [1/50], Loss: 4.6399
Epoch: [1/50], Loss: 4.6300
Epoch: [1/50], Loss: 4.6212
Epoch: [1/50], Loss: 4.6237
Epoch: [1/50], Loss: 4.6273
Epoch: [1/50], Loss: 4.6184
Epoch: [1/50], Loss: 4.6309
Epoch: [1/50], Loss: 4.6455
Epoch: [1/50], Loss: 4.6486
Epoch: [1/50], Loss: 4.6363
Epoch: [1/50], Loss: 4.6369
Epoch: [1/50], Loss: 4.6297
Epoch: [1/50], Loss: 4.6400
Epoch: [1/50], Loss: 4.6546
Epoch: [1/50]

In [95]:
with torch.no_grad():
    correct = 0
    total = 0
    for images,labels in test_dataloader:
        images,labels = images.to(device),labels.to(device)
        outputs = model(images)
        _,predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted==labels).sum().item()
    acc = 100 * correct/total
print(f'Accuracy on test data: {acc}')

Accuracy on test data: 1.4473898194828427
